In [2]:
import sys, os, time
#%load_ext autoreload
#%autoreload 2
import foundation as fd
import foundation.util as util
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision
import h5py as hf
import hashlib
from html.parser import HTMLParser
import urllib
import urllib.request
from urllib.error import URLError, HTTPError
import requests
from extraction import *
import redis

In [3]:
URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    #"list": "allpages",
    'allimit': 'max',
    #'titles': 'Exclamation mark',
    'titles': 'Double_factorial|Artificial_intelligence|AI ethics',
    'prop': "redirects",
    #"apfrom": "jungle", # continue
}

R = requests.get(url=URL, params=PARAMS)
DATA = R.json()

#print(DATA)

In [202]:
DATA.keys()

dict_keys(['query', 'continue'])

In [203]:
len(DATA['query']['allpages'])

KeyError: 'allpages'

In [205]:
len(DATA['query']['pages'])

3

In [204]:
DATA['query']['allpages']

KeyError: 'allpages'

In [206]:
DATA['query']['pages']

{'1164': {'ns': 0,
  'pageid': 1164,
  'redirects': [{'ns': 0, 'pageid': 1268, 'title': 'AI'},
   {'ns': 0, 'pageid': 83979, 'title': 'Machine intelligence'},
   {'ns': 0, 'pageid': 87949, 'title': 'AI ethics'},
   {'ns': 0, 'pageid': 87959, 'title': 'AI implications'},
   {'ns': 0, 'pageid': 246362, 'title': 'Artificially intelligent'},
   {'ns': 0, 'pageid': 524509, 'title': 'Artificial Intelligence'},
   {'ns': 0, 'pageid': 638220, 'title': 'The Artificial Intelligence'},
   {'ns': 0, 'pageid': 640610, 'title': 'Artificial intelligence (AI)'},
   {'ns': 0, 'pageid': 644536, 'title': 'Artificial Intelligence (AI)'},
   {'ns': 0, 'pageid': 886891, 'title': 'Machine understanding'}],
  'title': 'Artificial intelligence'},
 '507208': {'ns': 0, 'pageid': 507208, 'title': 'Double factorial'},
 '87949': {'ns': 0, 'pageid': 87949, 'title': 'AI ethics'}}

In [87]:
root = b'https://en.wikipedia.org/wiki/'

In [65]:
db = redis.Redis(port=6380)

In [69]:
all_inserted = db.smembers('articles')
len(all_inserted)

5202136

In [70]:
all_redirs = db.smembers('redirs')
len(all_redirs)

3063158

In [68]:
all_fails = db.smembers('failed')
len(all_fails)

5756

In [76]:
#fails = list(all_fails)
redirs = list(all_redirs)

In [94]:
name = redirs[2]
name

b'Shannon%27s_entropy'

In [95]:
url = root + name
url

b'https://en.wikipedia.org/wiki/Shannon%27s_entropy'

In [106]:
r = urllib.request.urlopen(url.decode())
r.geturl()

'https://en.wikipedia.org/wiki/Shannon%27s_entropy'

In [112]:
#r.read()

In [61]:
n.split('#')[0]

'FIFA_World_Cup_records'

In [43]:
alls = set(all_redirs)

In [44]:
full = [name.decode() for name in all_redirs if b'#' in name]

In [45]:
full[:10]

['FIFA_World_Cup_records#Goalscoring',
 'United_States_House_of_Representatives_elections_in_Arizona,_2016#District_8',
 '2018%E2%80%9319_UEFA_Champions_League_qualifying_phase_and_play-offs#Second_qualifying_round',
 'SR-25#Design',
 'Grand_Marais_Road#West_Grand_Boulevard',
 'Fijian_diplomatic_missions#High_Commissions',
 'Star_Trek:_The_Next_Generation_DC_comics#Modala_Imperative.2C_The',
 '2010_European_Men%27s_Handball_Championship_qualification_-_Group_5#Germany_v_Israel',
 'List_of_The_Bill_characters_(Q-Z)#Claire_Stanton',
 'White_Lady_(cocktail)#White_Lady']

In [46]:
full[0][:full[0].find('#')]

'FIFA_World_Cup_records'

In [47]:
cut = [name[:name.find('#')].encode() for name in full]

In [48]:
missing = [n for n in cut if n not in alls]
len(missing)

9331

In [49]:
missing[-30:]

[b'Minor_characters_from_The_Hitchhiker%27s_Guide_to_the_Galaxy',
 b'Sports_in_Bangladesh',
 b'Names_of_European_cities_in_different_languages:_Q%E2%80%93T',
 b'Glossary_of_terms_in_Hinduism',
 b'OPJHL_Standings_(1972-1981)',
 b'Art_Deco_theaters_of_the_Philippines',
 b'Characters_of_Crash_Bandicoot',
 b'List_of_minor_Star_Wars_characters',
 b'1940%E2%80%9349_Pacific_typhoon_seasons',
 b'Early_March_2012_tornado_outbreak',
 b'Copa_Libertadores_2009_preliminary_round',
 b'List_of_Joan_of_Arcadia_episodes',
 b'List_of_Pok%C3%A9mon_episodes_(seasons_14-current)',
 b'List_of_recurring_characters_from_The_Mighty_Boosh',
 b'The_Concert_For_Bangla_Desh',
 b'List_of_characters_from_The_Sopranos_%E2%80%93_Friends_and_Family',
 b'List_of_Terminator:_The_Sarah_Connor_Chronicles_minor_characters',
 b'Mitsubishi_Grunder',
 b'Uppiliyapuram_(ST)_(State_Assembly_Constituency)',
 b'Alternative_versions_of_Punisher',
 b'Citadel_Mall_(Colorado_Springs,_Colorado)',
 b'Colombian_Marine_Infantry',
 b'2006_W

In [50]:
len(alls)

1415003

In [4]:
db.keys()

[b'redirs', b'articles', b'failed']

In [5]:
db.scard('failed')

466

In [12]:
failed = db.smembers('failed')
failed = list(failed)
len(failed)

466

In [15]:
fails = {}

In [16]:
for i, x in enumerate(failed):
    if x not in fails:
        try:
            out = download_article_html_urllib(x)
            fails[x] = 0
        except URLError as e:
            fails[x] = e.code
        if i % 20 == 0:
            print(i+1)
len(failed), len(fails)

1
21
41
61
81
101
121
141
161
181
201
221
241
261
281
301
321
341
361
381
401
421
441
461


(466, 466)

In [17]:
codes = np.array([fails[k] for k in failed])

In [25]:
set(codes)

{0, 400, 404}

In [20]:
cs = {}
for x, c in fails.items():
    if c not in cs:
        cs[c] = []
    cs[c].append(x)

In [23]:
len(cs[400]), len(cs[404])

(73, 87)

In [27]:
o1, o2 = download_article_html_urllib, download_article_html_requests

In [28]:
res400 = [wiki_expand(x, o2) for x in cs[400]]

In [29]:
res404 = [wiki_expand(x) for x in cs[404]]

In [30]:
len(res400), len(res404)

(73, 87)

In [4]:
cs = {}

In [5]:
cs[404] = [b'Classification_in_the_Olympic_Games_Road_Race',
 b'Voivodeship_road_719_(Poland)',
 b'Department_of_Statistics,_University_of_Oxford',
 b'Oleksiivska',
 b'%CE%94%C2%B2_Tau',
 b'Urab%C3%A1_Basin',
 b'IF_Svea',
 b'Louisiana_Route_130',
 b'Coat_of_arms_of_the_Republic_of_Malawi',
 b'Dover_Minster',
 b'Greater_Toronto_Jaguars',
 b'B473_Lyr',
 b'Bundesstra%C3%9Fe_40',
 b'Frataraka_dynasty',
 b'Snaith_Priory',
 b'Fotheringhay_Nunnery',
 b'European_route_E981',
 b'Bus_Terminal_(Saint_Petersburg)',
 b'State_road_D330_(Turkey)',
 b'National_road_72_(Poland)',
 b'Timiryazevskaya_railway_station',
 b'Bundesstra%C3%9Fe_100',
 b'Headley_Priory',
 b'Bundesstra%C3%9Fe_472',
 b'First_flag_of_T%C3%A2y_S%C6%A1n_dynasty',
 b'Voivodeship_road_702_(Poland)',
 b'A9_Highway_(Kazakhstan)',
 b'R29_highway_(Azerbaijan)',
 b'Chester_Cathedral_Priory',
 b'Bundesstra%C3%9Fe_510',
 b'Temple_property',
 b'NS_Pup',
 b'Guajira_Offshore_Basin',
 b'15_LMi',
 b'Newland_Preceptory',
 b'State_road_D795_(Turkey)',
 b'Cratoxylum_hornschuchii',
 b'Port_Klang_International_Passenger_Terminal',
 b'Sinningthwaite_Priory',
 b'A17_Highway_(Kazakhstan)',
 b'Bangladesh_Honda_Private_Limited_(BHL)',
 b'DN1A',
 b'Line_5_(Athens_Tram)',
 b'Bundesstra%C3%9Fe_76',
 b'St_Anthony%27s_Monastery',
 b'Flag_of_Kherson_Oblast',
 b'European_route_E579',
 b'Coat_of_arms_of_the_Socialist_Republic_of_Croatia',
 b'Mountains_classification_in_the_Pro_Cycling_Challenge',
 b'National_road_17_(Poland)',
 b'Preston_Capes_Priory',
 b'Canterbury_%E2%80%94_Blackfriars',
 b'Bundesstra%C3%9Fe_64',
 b'General_Sarmiento_Railway',
 b'State_road_D180_(Turkey)',
 b'Louisiana_Route_150',
 b'Johanneshofs_IF',
 b'Kupchino_Railway_Platform',
 b'Great_Seal_of_the_State',
 b'Colombia_Deep_Pacific_Basin',
 b'National_road_92_(Poland)',
 b'Brantham_Hoard',
 b'Mordovian_Autonomous_Oblast',
 b'Leningradskaya_railway_station',
 b'Standard_of_the_Nguy%E1%BB%85n_dynasty_(1890%E2%80%931920)',
 b'Bundesstra%C3%9Fe_271',
 b'Ottringham_Priory',
 b'Voivodeship_road_138_(Poland)',
 b'M2_highway_(Kosovo)',
 b'Arms_of_Malawi',
 b'Order_of_Stara_Planina_(Bulgaria)',
 b'Petrovsko-Razumovskoye_railway_station',
 b'Stanney_Grange_(Cow_Worth_Grange)',
 b'M9_expressway_(Hungary)',
 b'Tehran_Metro_Line_6',
 b'Zambia_men%27s_national_field_hockey_team',
 b'19_Cep',
 b'Northampton_Austin_Friars',
 b'Blackwose_Priory',
 b'Bundesstra%C3%9Fe_43a',
 b'Flag_of_Communist_Party_of_Vietnam',
 b'Voivodeship_road_875_(Poland)',
 b'%C3%96rebro_BK',
 b'Rubezhivskyi_railway_station',
 b'Dover_Preceptory',
 b'A322_highway_(Russia)',
 b'Eastern_Cordillera_Basin']

In [10]:
cs[400] = [b'Frankfort,_Kentucky#Geography#Climate',
 b'University_of_Maryland,_Baltimore#University_of_Maryland#University_of_Maryland_School_of_Social_Work',
 b'Edward_Littleton_(died_1610)#Political_career#The_1597_election_scandal',
 b'Clustered_file_system##Shared-disk',
 b'Vikings#Culture#Everyday_life#Social_structure',
 b'List_of_World_Rally_Championship_records#Rallies#Closest_wins',
 b'Black_Canary_in_other_media#Live_Action#Arrow#Dinah_Drake',
 b'Chaotix#Members#Game_members#Espio_the_Chameleon',
 b'John_Bingham,_7th_Earl_of_Lucan#Personal_life##Marriage',
 b'Nicene_Creed#History#Comparison_between_creed_of_325_and_creed_of_381',
 b'Enlightenment_(spiritual)#Western_understanding#Experience',
 b'The_Duke_in_Darkness#Adaptations#1957_Australian_TV_adaptation',
 b'Jesus_in_Islam#Life##Second_coming',
 b'Ebisu,_Shibuya#Sightseeing_and_attractions#Yebisu_Garden_Place',
 b'List_of_Foreign_Ministers_of_Spain#First_Spanish_Republic_(1873-1874)#Ministers_of_State',
 b'The_Sun_(United_Kingdom)#Other_versions#The_Irish_Sun_and_The_Irish_Sun_on_Sunday',
 b'Predators_and_Prey#Part_I,_Harmonic_Divergence_(Issue#21)',
 b'Piala_Indonesia#History#Piala_Liga_(Amateur_Competition)',
 b'List_of_24_characters#24:_Live_Another_Day#CIA_London#Steve_Navarro',
 b'Johnston_Atoll#National_nuclear_weapon_test_site_1958-1963#Failures',
 b'The_Incredible_Hercules#"Assault_on_New_Olympus"_(Assault_on_New_Olympus_Prologue,#138-141)',
 b'2017%E2%80%9318_Basketball_Champions_League#Regular_season#Group_B',
 b'Minister_for_Home_Affairs_(Australia)#List_of_Ministers_for_Administrative#List_of_ministers_with_responsibilities_for_Australian_territories',
 b'Black_Fury_(comics)#Black_Fury#2_(Miss_Fury)',
 b"Commissar#Variants#People's_CommissarPeople",
 b'List_of_Cars_characters#"Junior"_#8',
 b'Education_in_Australia#School#Secondary',
 b'Global_surveillance_disclosures_(2013%E2%80%93present)#Reactions_of_political_leaders#Germany',
 b'GM_High_Feature_engine##LF1',
 b'2016%E2%80%9317_FIBA_Europe_Cup#Second_round#Group_K',
 b'Murder#Murder#Degrees_of_murder',
 b'Constitution_of_Estonia#History#First_Constitution_(1920\\xe2\\x80\\x931933)',
 b'List_of_24_characters#24:_Live_Another_Day#Open_Cell#Derrick_Yates',
 b'2018%E2%80%9319_UEFA_Champions_League##Group_stage',
 b'Bundeswehr#History#Cold_War_1955-1990',
 b'Ferrari_Dino_engine#V8#3.5',
 b'Military_discharge#United_States#Types_of_discharge#dishonorable',
 b'2005%E2%80%9306_Copa_del_Rey#Round_of_16#',
 b'List_of_24_characters#24:_Live_Another_Day#Drone_terrorist_plot#Ian_Al-Harazi',
 b'Denver_International_Airport_Automated_Guideway_Transit_System##History',
 b'List_of_24_characters#24:_Live_Another_Day#CIA_London#Erik_Ritter',
 b'Soul#Religious_views#Christianity',
 b'List_of_24_characters#24:_Live_Another_Day#Russian_Federation#Anatol_Stolnavich',
 b'Glenn_Miller_discography#Chesterfield_#2',
 b'KRIV_(TV)#Personalities#FOX_26_METEOROLOGISTS',
 b'1984_United_States_presidential_election#Republican_Party_candidates#Primaries',
 b'List_of_24_characters#24:_Live_Another_Day#CIA_London#Jordan_Reed',
 b'CITES#Appendices#Appendix_I',
 b'Second_law_of_thermodynamics#Description#Kelvin_statement',
 b'Trimix_(breathing_gas)#Mixes#Naming_conventions',
 b'Arc_lamp#Arc_lamp#History',
 b'13-Hydroxyoctadecadienoic_acid##Activities_of_13-HODEs',
 b'2011_Giro_d%27Italia#Stage_1_to_Stage_11#Death_of_Wouter_Weylandt',
 b'Pontiac_Solstice##GXP_(2007\\xe2\\x80\\x932009)',
 b'Asbestos_and_the_law#Future_of_civil_litigation#United_Kingdom',
 b'Turkic_mythology#Legends#Oghuz_legend',
 b'Android_version_history##v4.x_Ice_Cream_Sandwich',
 b'2010_Piala_Indonesia#Round_of_16#Group_11',
 b'List_of_24_characters#24:_Live_Another_Day#CIA_London#Gavin_Leonard',
 b'Direct3D#Direct3D_10#',
 b'2014%E2%80%9315_Copa_del_Rey#Round_of_32#First_Leg',
 b'Angela_Merkel#Early_political_career#Leader_of_the_Opposition',
 b'Copper_in_architecture#Applications#Wall_cladding',
 b'Batting_order_(baseball)##2',
 b'2014%E2%80%9315_Copa_del_Rey#Quarter-finals#Second_Leg',
 b'Chaotix#Members#Game_members#Vector_the_Crocodile',
 b'Hershey_Bears#Team_history#Modern_era',
 b'List_of_24_characters#United_Kingdom_officials,_employees,_and_associates#British_Prime_Minister_Alastair_Davies',
 b'2008_IAAF_World_Athletics_Final#Women#100_metres',
 b'Nikos_Karvelas#Personal_Discography#Albums',
 b'LaBeouf,_R%C3%B6nkk%C3%B6_%26_Turner##INTRODUCTIONS,_2015',
 b"List_of_Grey%27s_Anatomy_episodes#Season_5:2008-2009#Here's_to_the_Future",
 b'2008_Copa_Libertadores##Quarterfinals']

In [7]:
url = "https://en.wikipedia.org/wiki/{}".format(cs[404][0].decode())
url

'https://en.wikipedia.org/wiki/Classification_in_the_Olympic_Games_Road_Race'

In [8]:
url = "https://en.wikipedia.org/wiki/{}".format(cs[400][0].decode())
url

'https://en.wikipedia.org/wiki/Frankfort,_Kentucky#Geography#Climate'

In [12]:
url = "https://en.wikipedia.org/wiki/{}".format('Lourdinha_Bittencourt')
url

'https://en.wikipedia.org/wiki/Lourdinha_Bittencourt'

In [15]:
tests = [cs[404][0], cs[400][0], 'Lourdinha_Bittencourt']

In [18]:
cs[400][0]

b'Frankfort,_Kentucky#Geography#Climate'

In [17]:
wiki_expand(tests[1], download_article_html_requests)

([{'href': '/wiki/Kentucky', 'title': 'Kentucky'},
  {'href': '/wiki/Germany', 'title': 'Germany'},
  {'href': '/wiki/Frankfurt', 'title': 'Frankfurt'},
  {'href': '/wiki/List_of_capitals_in_the_United_States',
   'title': 'List of capitals in the United States'},
  {'href': '/wiki/List_of_cities_in_Kentucky',
   'title': 'List of cities in Kentucky'},
  {'href': '/wiki/Kentucky_State_Capitol', 'title': 'Kentucky State Capitol'},
  {'href': '/wiki/National_Register_of_Historic_Places_listings_in_Franklin_County,_Kentucky',
   'title': 'National Register of Historic Places listings in Franklin County, Kentucky'},
  {'href': '/wiki/Geographic_coordinate_system',
   'title': 'Geographic coordinate system'},
  {'href': '/wiki/U.S._state', 'title': 'U.S. state'},
  {'href': '/wiki/Kentucky', 'title': 'Kentucky'},
  {'href': '/wiki/List_of_counties_in_Kentucky',
   'title': 'List of counties in Kentucky'},
  {'href': '/wiki/Franklin_County,_Kentucky',
   'title': 'Franklin County, Kentucky'}

In [13]:
response = requests.get(url)

In [14]:
response.headers

{'Server': 'mw1268.eqiad.wmnet', 'Backend-Timing': 'D=523994 t=1545269260261582', 'Content-Encoding': 'gzip', 'X-Client-IP': '2607:4000:200:15:136:a724:89c3:f8b9', 'X-Cache-Status': 'hit-local', 'Cache-Control': 'private, s-maxage=0, max-age=0, must-revalidate', 'Set-Cookie': 'WMF-Last-Access=22-Dec-2018;Path=/;HttpOnly;secure;Expires=Wed, 23 Jan 2019 00:00:00 GMT, WMF-Last-Access-Global=22-Dec-2018;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Wed, 23 Jan 2019 00:00:00 GMT, GeoIP=US:WA:Seattle:47.66:-122.30:v4; Path=/; secure; Domain=.wikipedia.org', 'Server-Timing': 'cache;desc="hit-local"', 'Age': '3671', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="This is not a P3P policy! See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Via': '1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1), 1.1 varnish (Varnish/5.1)', 'Content-language': 'en', 'Content-Length': '14343', 'Date': 'Sat, 22 Dec 2018 05:43:32 GMT', 'X-Cache':

In [72]:
#response.text

In [73]:
response.status_code

200

In [ ]:
res404 = [wiki_expand(x) for x in cs[404]]